In [16]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp
np.set_printoptions(suppress=True)

Testing analytical versus numerical for different configurations

In [17]:
thetalist = np.array([0,-90,90,0,0,0])*np.pi/180
kr6 = KR6(thetalist)
print(np.isclose(tools.analyticalInverseKinematics(kr6.Tsb_poe)[2], kr6.Tsb_IK))

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [18]:
thetalist = np.array([0,0,0,0,0,0])
kr6 = KR6(thetalist)
print(np.isclose(tools.analyticalInverseKinematics(kr6.Tsb_poe)[2], kr6.Tsb_IK))

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [23]:
thetalist = np.array([45,-90,90,10,10,10])
kr6 = KR6(thetalist)
print(np.isclose(tools.analyticalInverseKinematics(kr6.Tsb_poe)[2], kr6.Tsb_IK))
print(tools.analyticalInverseKinematics(kr6.Tsb_poe)[2])
print(kr6.Tsb_dh)

[[ 0.30230201  0.65490275 -0.69261525  0.07125983]
 [ 0.34982431 -0.75213437 -0.55849517 -0.24985501]
 [-0.88669975 -0.07345944 -0.45647263  0.80525067]
 [ 0.          0.          0.          1.        ]]
[[ 0.30230201 -0.65490275 -0.69261525  0.07125983]
 [-0.34982431 -0.75213437  0.55849517 -0.16049578]
 [-0.88669975  0.07345944 -0.45647263  0.80525067]
 [ 0.          0.          0.          1.        ]]
[[ True False  True  True]
 [False  True False False]
 [ True False  True  True]
 [ True  True  True  True]]
